# SpinToolkit

Demo code for Spin wave theory for complex spin orders I: linear spin wave, generalized linear spin wave, multi-magnon continuum and finite-temperature effects_

Authors: Lei Xu, Xiaojian Shi, Yangjie Jiao, and [Zhentao Wang](https://orcid.org/0000-0001-7442-2933)

In this demo, we compute the finite-$T$ multi-boson excitation of the triangular lattice spin-$1$ XXZ model with single-ion anisotropy

## Prerequisites

import the modules and set the number of threads for parallelling

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from joblib import Parallel, delayed
import SpinToolkit_py as sptk

In [ ]:
# check the system information and choose the number of threads accordingly
sptk.print_system_info()

n_jobs = 20 # set manually
# n_jobs = n_jobs   # use the maximum number of threads available

## Define lattice and hamiltonian

- triangular lattice with a 3-sublattice supercell compatible with 120$^\circ$ order

- spin-$1$ XXZ model with single-ion anisotropy

- use "SU(N)" mode

In [ ]:
latt = sptk.lattice(filename="triangular_K.toml")
Nsites = latt.Nsites()

In [ ]:
J =  1.0         # nearest-neighbor bilinear exchange
D = -5.0         # quadratic exchange
spin = 1.0

# consider Hamiltonian for different magnetic field

def generate_Hamiltonian(H : float, verbose: bool = True) -> sptk.model_spin:
    hamiltonian = sptk.model_spin(S = spin, mode = "SU(N)", lattice = latt, verbose = verbose)

    for site_i in range(Nsites):
        coor_i, sub_i = latt.site2coor(site = site_i)
        coor0_i, Mi   = latt.coor2supercell0(coor = coor_i)
        xi = coor_i[0]
        yi = coor_i[1]

        # 1st neighbor terms
        coor_j      = [xi + 1, yi]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, J),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        coor_j      = [xi, yi + 1]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, J),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        coor_j      = [xi + 1, yi + 1]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, J),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        hamiltonian.add_site_Amatrix(D = sptk.Vec3(0, 0, D),
                                      site = site_i)
        hamiltonian.add_zeeman(h = sptk.Vec3(0, 0, H),
                               site = site_i)

    hamiltonian.simplify(verbose = verbose)
    hamiltonian.build_mc_list(verbose = verbose)
    return hamiltonian

## Define the minimization (minimize from several independent initial random guesses, then pick the lowest-energy solution)

In [ ]:
def minimize(hamiltonian: sptk.model_spin, verbose: bool=True) -> tuple[list[complex], float]:
    if verbose:
        print("Optimizing ground state", end = "... ", flush = True)
    start        = time.perf_counter()
    Z_min, f_min = hamiltonian.optimize_spins_SUN(total_seeds = 20) # more seeds -> more likely to hit global minimum
    end          = time.perf_counter()
    if verbose:
        print(f"{end - start}s")
        print()

        print(f"f_min (global): {f_min}")
        for i in range(Nsites):
            print(f"site-{i}: {Z_min[i]}")

    return Z_min, f_min

## Compute the model in full polarized phase

In [ ]:
hamiltonian = generate_Hamiltonian(H = 15.0)
Z_min, f_min = minimize(hamiltonian, verbose = True)
mx, my, mz = sptk.spins_magnetization(Z = Z_min)
print(f"Magnetization: mx = {mx}, my = {my}, mz = {mz}")

## Initialize the GLSW calculation

In [ ]:
hamiltonian.init_GLSW(Z = Z_min)

## Define a $k$-space trajectory, a list of $\omega$, Gaussian broadening factor $\sigma$, and temperature $T$ for computing $\mathcal{S}^{ab}({\bm k}, \omega)$

In [ ]:
kc = sptk.k_cut(lattice = latt, density = 50)
kc.add_k(k_new = [0.0, 0.0])              # Gamma
kc.add_k(k_new = [0.5, 0.0])              # M
xtic_M = kc.len_list[-1]                  # x-axis position for M-point (for plotting)
kc.add_k(k_new = [1.0 / 3.0, 1.0 / 3.0])  # K
xtic_K = kc.len_list[-1]                  # x-axis position for K-point (for plotting)
kc.add_k(k_new = [0.0, 0.0])              # Gamma
total_k = kc.size()

ω_min = -30.0
ω_max =  30.0
num_ω =  1000
ω_list = np.arange(ω_min, ω_max, (ω_max - ω_min) / num_ω, dtype = np.float64).tolist()

# Gaussian broadening factor
FWHM  = 0.5
sigma = FWHM / 2.35482

# Temperature
T = 2 * J


## Compute finite-$T$ 0- and 1-boson DSSF $\mathcal{S}_{0}^{ab}({\bm k},\omega) + \mathcal{S}_{1}^{ab}({\bm k},\omega) $

In [ ]:

omega_k  = []
Sxx      = []
Syy      = []
Szz      = []
SxyPyx_R = []
SyzPzy_R = []
SzxPxz_R = []
SxyMyx_I = []
SyzMzy_I = []
SzxMxz_I = []

def process_Sk(index_k):
    k = kc.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        return sptk.DSSF_GLSW(model = hamiltonian, T = T,
                              k = k, omega_list = ω_list,
                              eval_1boson = True, maxeval_2boson = 0,
                              maxeval_3boson = 0, maxeval_0boson = 100000,
                              broadening = "Gaussian", sigma_or_eta = sigma,
                              epsilon = 1.0e-4)
    except Exception as e:
        print(f"error: {e}, k = {k}")

start = time.perf_counter()
results_Sk = Parallel(n_jobs = n_jobs, prefer = "threads")(
    delayed(process_Sk)(index_k) for index_k in range(total_k))
omega_k, Sxx, Syy, Szz, \
SxyPyx_R, SyzPzy_R, SzxPxz_R, \
SxyMyx_I, SyzMzy_I, SzxMxz_I = zip(*results_Sk)
print()
end = time.perf_counter()
print(f"Used {(end - start) / 60.0:.6f}m")

In [ ]:
_, num_disp = np.array(omega_k).shape
x = kc.len_list

INSdatafile = f"INS_multiboson_T{T}K_triangular.txt"
Dispdatafile = f"Disp_multiboson_T{T}K_triangular.txt"

# save data
axis_X, axis_Y = np.meshgrid(x, ω_list)
Intensity = (np.array(Sxx) + np.array(Syy) + np.array(Szz)).transpose()

np.savetxt(INSdatafile,
         np.column_stack((axis_X.flatten(), axis_Y.flatten(), Intensity.flatten())),
           header="#(1)\t(2)\t(3)\n#k\tomega\tIntensity")

np.savetxt(Dispdatafile, np.column_stack((x, np.array(omega_k))),
           header="\t".join([f"({i})" for i in range(num_disp + 1)] + ["\nk"] + [f"omega_{i+1}" for i in range(num_disp)]),
           comments='#')

# read data
# num_rows = len(ω_list)
# num_cols = len(x)
# axis_X, axis_Y, Intensity = np.loadtxt(INSdatafile, unpack=True)
# axis_X = axis_X.reshape((num_rows, num_cols))
# axis_Y = axis_Y.reshape((num_rows, num_cols))
# Intensity = Intensity.reshape((num_rows, num_cols))

# data = np.loadtxt(Dispdatafile)
# omega_k = data[:, 1:]


## Plot 0- and 1-boson DSSF $\mathcal{S}({\bm k}, \omega) \equiv \mathcal{S}_{0}^{xx}({\bm k}, \omega) + \mathcal{S}_{0}^{yy}({\bm k}, \omega) + \mathcal{S}_{0}^{zz}({\bm k}, \omega) + \mathcal{S}_{1}^{xx}({\bm k}, \omega) + \mathcal{S}_{1}^{yy}({\bm k}, \omega) + \mathcal{S}_{1}^{zz}({\bm k}, \omega)$

In [ ]:
plt.rcParams['figure.figsize'] = (4, 3)
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['font.size'] = 15

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,newtxtext,newtxmath,bm}'
plt.rcParams['font.family'] = 'TeX Gyre Termes'

fig, ax = plt.subplots()

vmin = 0.001
vmax = 1
cmap = colors.LinearSegmentedColormap.from_list("white_to_blue", [(1, 1, 1), (0, 0, 1)], N = 256)
# logscale
imag = ax.pcolormesh(axis_X, axis_Y, Intensity, cmap = cmap, shading = "auto", norm=colors.LogNorm(vmin=vmin, vmax=vmax))

# uncomment the following line if you like to overplot the 1-boson dispersion
# ax.plot(x, np.array(omega_k), 'r--')

xmin = x[0]
xmax = x[-1]
ymax = 24
ymin = -24
ax.set_xlim(x[0], x[-1])
ax.set_ylim(ymin, ymax)
ax.set_xticks([x[0], xtic_M, xtic_K, x[-1]], [r"$(0,0)$", r"$(\frac{1}{2},0)$", r"$(\frac{1}{3},\frac{1}{3})$", r"$(0,0)$"])
ax.set_yticks([-24, -12, 0, 12, 24])
ax.set_xlabel(r"$\bm{k}$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(0.66 * xmax, -0.7 * ymax, r'$\mathcal{S}_{0} + \mathcal{S}_{1}$', fontsize = 15)
ax.text(0.48 * xmax, -0.9 * ymax, r'$10^{-3}$', fontsize = 15)
ax.text(0.95 * xmax, -0.9 * ymax, fr'${vmax}$', fontsize = 15)

cax = ax.inset_axes([0.61 * xmax, -0.9 * ymax, 0.32 * (xmax-xmin), 0.05 * (ymax - ymin)], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0.001, 0.01, 0.1, 1])
cbar.set_ticklabels(['', '', '', ''])
cbar.ax.tick_params(which='both', direction = 'in')

plt.savefig(f"Fig_multiboson_T{T}K_triangular.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()

## Compute finite-$T$ 0- and 1-boson DSSF $\mathcal{S}_{0}^{ab}({\bm k},\omega) + \mathcal{S}_{1}^{ab}({\bm k},\omega)$ at ${\rm \Gamma}$ point under different magnetic fields

In [ ]:
# create the field H_list
H_min = 10
H_max = 16
H_list = np.arange(H_min, H_max, 0.05, dtype = np.float64).tolist()
total_H = len(H_list)

In [ ]:
omega_k  = []
Sxx      = []
Syy      = []
Szz      = []
SxyPyx_R = []
SyzPzy_R = []
SzxPxz_R = []
SxyMyx_I = []
SyzMzy_I = []
SzxMxz_I = []

def process_SH(index_H):
    k = [0.0, 0.0]  # Gamma point
    H = H_list[index_H]
    hamiltonian = generate_Hamiltonian(H = H, verbose=False)
    Z_min, f_min = minimize(hamiltonian, verbose = False)
    hamiltonian.init_GLSW(Z = Z_min)

    if index_H % sptk.round2int(total_H / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:

        return sptk.DSSF_GLSW(model = hamiltonian, T = T,
                              k = k, omega_list = ω_list,
                              eval_1boson = True, maxeval_2boson = 0,
                              maxeval_3boson = 0, maxeval_0boson = 100000,
                              broadening = "Gaussian", sigma_or_eta = sigma,
                              epsilon = 1.0e-4)
    except Exception as e:
        print(f"error: {e}, H = {H}")

start = time.perf_counter()
results_SH = Parallel(n_jobs = n_jobs, prefer = "threads")(
    delayed(process_SH)(index_H) for index_H in range(total_H))
omega_k, Sxx, Syy, Szz, \
SxyPyx_R, SyzPzy_R, SzxPxz_R, \
SxyMyx_I, SyzMzy_I, SzxMxz_I = zip(*results_SH)
print()
end = time.perf_counter()
print(f"Used {(end - start) / 60.0:.6f}m")

In [ ]:
_, num_disp = np.array(omega_k).shape
x = H_list

INSdatafile = f"INS_field_multiboson_T{T}K_triangular.txt"
Dispdatafile = f"Disp_field_multiboson_T{T}K_triangular.txt"

axis_X, axis_Y = np.meshgrid(x, ω_list)
Intensity = (np.array(Sxx) + np.array(Syy) + np.array(Szz)).transpose()

# save data
np.savetxt(INSdatafile, np.column_stack((axis_X.flatten(), axis_Y.flatten(), Intensity.flatten())),
           header="#(1)\t(2)\t(3)\n#H\tomega\tIntensity")

np.savetxt(Dispdatafile, np.column_stack((x, np.array(omega_k))),
           header="\t".join([f"({i})" for i in range(num_disp + 1)] + ["\nH"] + [f"omega_{i+1}" for i in range(num_disp)]),
           comments='#')

# read data
# num_rows = len(ω_list)
# num_cols = len(x)
# axis_X, axis_Y, Intensity = np.loadtxt(INSdatafile, unpack=True)
# axis_X = axis_X.reshape((num_rows, num_cols))
# axis_Y = axis_Y.reshape((num_rows, num_cols))
# Intensity = Intensity.reshape((num_rows, num_cols))

# data = np.loadtxt(Dispdatafile)
# omega_k = data[:, 1:]



## Plot field induced 0- and 1-boson DSSF $\mathcal{S}({\bm k}, \omega) \equiv \mathcal{S}_{0}^{xx}({\bm k}, \omega) + \mathcal{S}_{0}^{yy}({\bm k}, \omega) + \mathcal{S}_{0}^{zz}({\bm k}, \omega) + \mathcal{S}_{1}^{xx}({\bm k}, \omega) + \mathcal{S}_{1}^{yy}({\bm k}, \omega) + \mathcal{S}_{1}^{zz}({\bm k}, \omega)$

In [ ]:
plt.rcParams['figure.figsize'] = (4, 3)
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['font.size'] = 15

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,newtxtext,newtxmath,bm}'
plt.rcParams['font.family'] = 'TeX Gyre Termes'

fig, ax = plt.subplots()

vmin = 0.001
vmax = 1
cmap = colors.LinearSegmentedColormap.from_list("white_to_blue", [(1, 1, 1), (0, 0, 1)], N = 256)

imag = ax.pcolormesh(axis_X, axis_Y, Intensity, cmap = cmap, shading = "auto", norm=colors.LogNorm(vmin=vmin, vmax=vmax))

# uncomment the following line if you like to overplot the 1-boson dispersion
ax.plot(x, np.array(omega_k), 'r--', linewidth=1)

xmin = H_list[0]
xmax = H_list[-1]
ymin = -24
ymax = 24
ax.set_xlim(x[0], x[-1])
ax.set_ylim(ymin, ymax)
ax.set_xticks([10, 12, 14 ,16])
ax.set_yticks([-24, -12, 0, 12, 24])
ax.set_xlabel(r"$H/J$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(0.885 * xmax, -0.7 * ymax, r'$\mathcal{S}_{0} + \mathcal{S}_{1}$', fontsize = 15)
ax.text(0.81 * xmax, -0.9 * ymax, r'$10^{-3}$', fontsize = 15)
ax.text(0.99 * xmax, -0.9 * ymax, fr'${vmax}$', fontsize = 15)

cax = ax.inset_axes([0.86 * xmax, -0.9 * ymax, 0.32 * (xmax-xmin), 0.05 * (ymax - ymin)], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0.001, 0.01, 0.1, 1])
cbar.set_ticklabels(['', '', '', ''])
cbar.ax.tick_params(which='both', direction = 'in')

plt.savefig(f"Fig_field_multiboson_T{T}K_triangular.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()